# Deploying Machine Learning Models using ksonnet and Ambassador on Google Kubernetes
## Experimental


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - You need a running GCP cluster with kubernetes>1.8 with kubectl configured to use.
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

## Create Namespace

In [1]:
!kubectl create namespace seldon

namespace "seldon" created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_kubeflow-testing-213813_europe-west1-b_cluster-1" modified.


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [3]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")

clusterrolebinding "my-cluster-admin-binding" created


In [4]:
!ks init my-ml-deployment

INFO Using context "gke_kubeflow-testing-213813_europe-west1-b_cluster-1" from kubeconfig file "/home/clive/.kube/config" 
INFO Creating environment "default" with namespace "seldon", pointing to "version:v1.9.7" cluster at address "https://146.148.119.24" 
INFO Generating ksonnet-lib data at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/lib/ksonnet-lib/v1.9.7' 


In [5]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --withAmbassador=true --withRbac=true --namespace=seldon

INFO Retrieved 7 files                            
INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/seldon-core.jsonnet' 


In [6]:
!cd my-ml-deployment && \
      ks apply default

INFO Applying services seldon.seldon-core-ambassador 
INFO Creating non-existent services seldon.seldon-core-ambassador 
INFO Applying clusterroles seldon-crd             
INFO Creating non-existent clusterroles seldon-crd 
INFO Applying roles seldon.seldon-local           
INFO Creating non-existent roles seldon.seldon-local 
INFO Applying rolebindings seldon.seldon          
INFO Creating non-existent rolebindings seldon.seldon 
INFO Applying clusterrolebindings seldon.seldon   
INFO Creating non-existent clusterrolebindings seldon.seldon 
INFO Applying rolebindings seldon.ambassador      
INFO Creating non-existent rolebindings seldon.ambassador 
INFO Applying serviceaccounts seldon.seldon       
INFO Creating non-existent serviceaccounts seldon.seldon 
INFO Applying services seldon.seldon-core-redis   
INFO Creating non-existent services seldon.seldon-core-redis 
INFO Applying customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Creating non-existent customre

In [7]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [8]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

## Create Seldon Deployment

In [9]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [10]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-classifier-0 replicas:1 replicasAvailable:1] map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1]]]

## Get predictions

In [11]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [12]:
rest_request_ambassador("seldon-model",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "mgfh2bvpdjtr6otkebtvvt9ldb",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09262920045371324]
    }
  }
}


#### gRPC Request

In [13]:
 grpc_request_ambassador("seldon-model",API_AMBASSADOR)

meta {
  puid: "qak4oag1rnpivslrv3hsv6v17a"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0824669435792235
  }
}



## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [17]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

service "example-auth" created
deployment "example-auth" created


** Need to wait until running before adding Ambassador config **

In [18]:
!kubectl get pods -n seldon

NAME                                                    READY     STATUS              RESTARTS   AGE
example-auth-54df78c744-dw9vc                           0/1       ContainerCreating   0          3s
seldon-core-ambassador-7fb4575f6b-57g9j                 2/2       Running             0          3m
seldon-core-redis-54f855dd66-c96xw                      1/1       Running             0          3m
seldon-core-seldon-cluster-manager-5649f8cddf-bkvtw     1/1       Running             0          3m
test-deployment-example-classifier-0-678f5694bd-cvs2v   1/1       Running             0          1m
test-deployment-example-svc-orch-5df757ff4-zlv8s        1/1       Running             0          1m


In [19]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

service "example-auth" configured


Show failed request when auth is running

In [21]:
rest_request_ambassador("seldon-model",API_AMBASSADOR)

401



Show successful request with auth

In [22]:
rest_request_ambassador_auth("seldon-model","username","password",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "1as8t5mh7eh86v2ujj89g07p18",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.07221362017270441, 0.07728004249492534]
    }
  }
}


# Tear down

In [ ]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

In [ ]:
!kubectl delete -f resources/model.json -n seldon

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment